In [2]:
from collections import defaultdict
import numpy as np

In [202]:
# CSA helpers

def search_next_departure(stop_departures, time):
    '''
        Search the first departure after moment time in the same stop through binary search in the list of S[stop]
    '''
    start = 0
    end = len(stop_departures) - 1

    next_departure_index = -1
    while start <= end:
        mid = (start + end) // 2;

        # Search in the right side
        if (stop_departures[mid]['departure_time'] < time):
            start = mid + 1

        # Search in the left side
        else:
            next_departure_index = mid
            end = mid - 1
     
    
    # Search is always successful because of the last np.inf departure
    next_profile_entry = stop_departures[next_departure_index]
        
    return next_departure_index, next_profile_entry

def shift(vector):
    # TODO: include max for earliest arrival
    new_vector = vector.copy()
    new_vector[1:] = vector[:-1]
    new_vector[0] = np.inf
    return new_vector

def minimize(vector1, vector2):
    return np.minimum(vector1, vector2)

def minimize_with_exits(vector_with_exits, tau_c, connection):
    indices_to_change = np.where(tau_c < vector_with_exits[0])[0]
    
    new_arrival_times = vector_with_exits[0].copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_exits = vector_with_exits[1].copy()
    for index in indices_to_change:
        new_exits[index] = connection
    print(new_exits)
    return (new_arrival_times, new_exits)
    
def minimize_with_stops(old_arrivals, old_connections, tau_c, connection):
    indices_to_change = np.where(tau_c < old_arrivals)[0]
    
    new_arrival_times = old_arrivals.copy()
    new_arrival_times[indices_to_change] = tau_c[indices_to_change]
    new_connections = old_connections.copy()
    for index in indices_to_change:
        new_connections[index] = connection
    return (new_arrival_times, new_connections)

In [205]:
def CSA(timetable, source_stop, target_stop, min_departure_time, max_arrival_time, max_connections):
    '''
        Implementation of Pareto Connection Scan profile algorithm with interstop footpaths (https://arxiv.org/pdf/1703.05997.pdf)
    '''
    # timetable
    stops, connections, trips, footpaths = timetable
    S = defaultdict(lambda: [{'departure_time': np.inf, 'arrival_times': np.ones(max_connections) * np.inf, 'enter_connections': [None] * max_connections, 'exit_connections': [None] * max_connections}]) # entries of the type       stop: [(departure_time, [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection, [outgoing_connections])]
    T = defaultdict(lambda: {'arrival_times': np.ones(max_connections) * np.inf, 'exit_connections': [None] * max_connections})   # trip: [arrival_time_with_1_connection, ..., arrival_time_with_max_connections], incoming_connection
    
    for connection in connections:
        c_dep_stop, c_arr_stop, c_dep_time, c_arr_time, c_trip = connection
        
        # PHASE 1: FIND tau_c (arrival times given this connection)
        
        # Arrival times by walk from c_arr_stop 
        # TODO (right now it is just a place-holder)
        tau_1 = np.ones(max_connections) * c_arr_time if c_arr_stop == target_stop else np.inf
        
        # Arrival times by continuing on the same trip
        tau_2 = T[c_trip]['arrival_times']
        
        # Arrival times by moving to another trip
        # TODO add walk time
        tau_3 = search_next_departure(S[c_arr_stop], c_arr_time)[1]['arrival_times']
        tau_3 = shift(tau_3)
        
        # Find minimum per number of changes between tau_1, tau_2 and tau_3
        tau_c = minimize(minimize(tau_1, tau_2), tau_3)
        if c_dep_stop == 'y':
            print(tau_1, tau_2, tau_3, tau_c)
        # PHASE 2: UPDATE S
        
        # arrivals of earliest departure after c_dep_time from stop c_dep_stop
        next_departure_index, next_profile_entry = search_next_departure(S[c_dep_stop], c_dep_time)
        y = next_profile_entry['arrival_times']
        y_departure_connections = next_profile_entry['enter_connections']
        new_min_arrivals, new_enter_connections = minimize_with_stops(y, y_departure_connections, tau_c, connection)
        if not np.array_equal(y, new_min_arrivals):
            # add another pair before the next one
            S[c_dep_stop].insert(next_departure_index, {'departure_time': c_dep_time, 'arrival_times': new_min_arrivals, 'enter_connections': new_enter_connections, 'exit_connections': T[c_trip]['exit_connections']})
        
        # PHASE 3: UPDATE T
        new_min_arrivals, new_exit_connections = minimize_with_stops(T[c_trip]['arrival_times'], T[c_trip]['exit_connections'], tau_c, connection)
        T[c_trip]['arrival_times'] = new_min_arrivals
        T[c_trip]['exit_connections'] = new_exit_connections
        
    return S

In [206]:
paper_conn = [
    ("y", "t", 10, 11, "1"),
    ("z", "t", 9, 12, "2"),
    ("x", "t", 8, 13, "3"),
    ("x", "y", 8, 9, "4"),
    ("s", "z", 7, 8, "5"),
    ("s", "x", 6, 7, "6"),
    ("s", "t", 5, 14, "7")
]
paper_timetable = (None, paper_conn, None, None)

CSA(paper_timetable, "s", "t", 4, 15, 3)

[11. 11. 11.] [inf inf inf] [inf inf inf] [11. 11. 11.]


defaultdict(<function __main__.CSA.<locals>.<lambda>()>,
            {'t': [{'departure_time': inf,
               'arrival_times': array([inf, inf, inf]),
               'enter_connections': [None, None, None],
               'exit_connections': [None, None, None]}],
             'y': [{'departure_time': 10,
               'arrival_times': array([11., 11., 11.]),
               'enter_connections': [('y', 't', 10, 11, '1'),
                ('y', 't', 10, 11, '1'),
                ('y', 't', 10, 11, '1')],
               'exit_connections': [None, None, None]},
              {'departure_time': inf,
               'arrival_times': array([inf, inf, inf]),
               'enter_connections': [None, None, None],
               'exit_connections': [None, None, None]}],
             'z': [{'departure_time': 9,
               'arrival_times': array([12., 12., 12.]),
               'enter_connections': [('z', 't', 9, 12, '2'),
                ('z', 't', 9, 12, '2'),
                ('z', 't', 

In [ ]:
def extract_journey(S, source_stop, source_time, changes):
    profiles_source_stop = S[source_stop]
    first_departure_index, _ = search_next_departure(profiles_source_stop, source_time)
    
    for i in range(first_departure_index, len(profiles_source_stop)):
        profile_entry = profiles_source_stop[i]
        enter_connection = profile_entry['enter_connection'][changes]
        if enter_connection
    for departure_time, profile_entry in S.items():
        if source_dep_time > departure_time: 
            pass
        for 